<a href="https://colab.research.google.com/github/BrenoMilagres/DataScienceProjetos/blob/master/WebScrapingBrasileirao2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Carregando módulo que possibilita utilizar um comando "mágico" para programar em R no google colab

In [ ]:
%reload_ext rpy2.ipython

#### Importando bibliotecas 

In [ ]:
%%R
library(stringr)
library(rvest)
library(glue)

### Acessando url do site da cbf 

In [ ]:
%%R
url <- glue("https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a")


#### Através da inspeção do Html do site, é possível notar que a classe aside-rodadas engloba todos os dados da tabela de jogos, então é necessário fazer a leitura desses dados utilizando os pacotes rvest e xml2.

In [ ]:
%%R
# Lendo classe html e alocando em resultado
resultados <- url %>% 
  read_html() %>% 
  html_nodes(".aside-rodadas")

head(resultados)

{xml_nodeset (1)}
[1] <aside class="aside-rodadas swiper-container"><div class="swiper-wrapper" ...


#### Extraindo informações que são pertinentes da tabela e estruturando-as para análises posteriores. Os campos que estamos buscando são:

>Time Casa: As classes que englobam as siglas dos times da casa são pull-left e time-sigla;

>Time Fora: para o time de fora, as classes são pull-right e time-sigla;

>Placar: é agrupado pela classe partida-horario.

In [ ]:
%%R
#Extraindo informações de time da casa
casa <- resultados %>% 
  html_nodes(".pull-left .time-sigla") %>% 
  html_text()

#Extraindo informações de time de fora
fora <- resultados %>% 
  html_nodes(".pull-right .time-sigla") %>% 
  html_text()

#Extraindo informações de time do placar
placar <-  resultados %>% 
            html_nodes(".partida-horario") %>%
             html_text() %>%
               str_extract("[0-9]{1}\ x\ [0-9]{1}")

#Gerando valores de rodadas    
rodada <- 0:(length(placar)-1) %/% 10 + 1

#Estruturando dados coletados em dataframe
df <- data.frame(cbind(rodada = rodada,
                    casa = casa,
                    placar = placar,
                    fora = fora,
                    ano = rep(2020,length(rodada) ) ) )

df

    rodada casa placar fora  ano
1        1  PAL   <NA>  VAS 2020
2        1  GOI   <NA>  SAO 2020
3        1  FOR  0 x 2  ATH 2020
4        1  COR  0 x 1  INT 2020
5        1  SPO  3 x 2  CEA 2020
6        1  FLA  0 x 1  ATL 2020
7        1  SAN  1 x 1  RED 2020
8        1  GRE  1 x 0  FLU 2020
9        1  BOT  1 x 2  BAH 2020
10       1  COR  0 x 0  ATL 2020
11       2  ATH  2 x 1  GOI 2020
12       2  RED  1 x 1  BOT 2020
13       2  ATL  3 x 2  COR 2020
14       2  ATL  3 x 0  FLA 2020
15       2  BAH  1 x 0  COR 2020
16       2  CEA  1 x 1  GRE 2020
17       2  FLU  1 x 1  PAL 2020
18       2  SAO  1 x 0  FOR 2020
19       2  INT  2 x 0  SAN 2020
20       2  VAS  2 x 0  SPO 2020
21       3  GRE  0 x 0  COR 2020
22       3  COR  0 x 1  FLA 2020
23       3  PAL  1 x 1  GOI 2020
24       3  ATL  2 x 0  CEA 2020
25       3  VAS  2 x 1  SAO 2020
26       3  BAH  2 x 1  RED 2020
27       3  FLU  2 x 1  INT 2020
28       3  ATL  1 x 1  SPO 2020
29       3  FOR  0 x 0  BOT 2020
30       3

#### Escrevendo um arquivo csv atrvés do dataframe gerado.

In [ ]:
%%R
write.csv(df, "Brasileirao_2020.csv", col.names = TRUE, row.names = FALSE, sep = ",")